In [1]:
import numpy as np 


# Task 1 (2 points)
Assume that now the price of stock is 200. In the next period price can be either 220 or 180, and you estimate
probability of an upmove as 0.7. Risk-free rate per period is 3%.
Using money-market account with starting value of 1.0 and the stock, construct a replicating portfolio for an
option with payoffs 20 and 0 in the up and down states, respectively, and price that option. Return as output a
numpy array with stock position, money-market position and option price. Beware of the order of variables in
output!

In [12]:
def task1(S0, Su, Sd, Pu, Pd, r, p=0.7):
 """
 inputs:
 S0 = current stock price,
 Su = stock price the next period in the up state,
 Sd = stock price the next period in the down state,
 Pu = payoff in the up state
 Pd = payoff in the down state
 r = interest rate per period as decimal (that is, 0.03 for 3%)
 p = probability of an up-move (0.7 by default)
 returns:
 numpy array with stock position, money-market position and option price. 
 Beware of the order of variables in output!
 """
 ans = None

 X = np.array([[Su, Sd],
 [1+r, 1+r]]) # payoff matrix
 payoffv = np.array([Pu, Pd]) # payoffs vector
 ptf = np.linalg.inv(X.T)@payoffv # delta and B 
 prcv = np.array([S0,1])
 prc = ptf@prcv
 ans = np.append(ptf,prc)
 ### END SOLUTION
 print(ptf)
 return ans

In [13]:
S0, Su, Sd, Pu, Pd, r = 200, 220, 180, 20, 0, 0.03
ans = task1(S0, Su, Sd, Pu, Pd, r)
ans

[  0.5        -87.37864078]


array([  0.5       , -87.37864078,  12.62135922])

# Task 2 (2 points)
A stock price is currently 100 USD. Over each of the next two periods (six months each) it is expected to go up
by 10% or down by 10%, and you estimate probability of an up-move as 0.7 for each period. The risk-free
interest rate is 8% p.a. with continuous compounding. Use risk-neutral valuation approach.
What are the prices of a European call and put options both with a strike price of 100 USD and expiring two
periods from now

In [43]:
def task2(S0 = 100, up = 1.1, dn = 0.9, r = 0.08, K = 100, dt = 0.5, p = 0.7):
 """
 inputs:
 S0 = initial stock price,
 up = growth factor for up state (here up = 1.1),
 dn = growth factor for down state (here dn = 0.9),
 r = risk free interest rate in decimals (i.e., 0.08 for 8%), with continuous com
 K = strike price,
 dt = period length in years (default = 0.5 years)
 p = probability of an up-move (0.7 by default)
 returns:
 numpy array with prices of European call and put options both with a strike of 1
 Beware of the order of variables in output!
 """
 ans = None
 ### BEGIN SOLUTION
 rf = np.exp(r * dt) # riskfree asset per period
 # risk-neutral probability of an up-move
 q = (rf - dn) / (up - dn) 
 qall = np.array([q**2, 2*q*(1-q), (1-q)**2])
 # calculate stock prices
 Suu = S0*up**2
 Sud = S0*up*dn # the tree is recombining so that up-down 
 #and down-up result in the same state
 Sdd = S0*dn**2
 # calculate option payoffs
 C2 = np.array([max(0,Suu-K), max(0,Sud-K), max(0,Sdd-K)])
 P2 = np.array([max(0,-Suu+K), max(0,-Sud+K), max(0,-Sdd+K)])
 # calculate option prices as discounted expectations
 C = C2@qall/rf**2
 P = P2@qall/rf**2
 ans = np.array([C,P])
 # END SOLUTION
 print(P)
 return ans
 
S0, up, dn, r, K, dt, p = 100, 1.12, 0.9, 0.08, 100, 0.5, 0.1

ans = task2()

ans

1.9208409406350113


array([9.6092063 , 1.92084094])

In [53]:
def task2a(S0 = 100, up = 1.1, dn = 0.9, r = 0.08, K = 100, dt = 0.5, p = 0.7):
    rf = np.exp(r * dt)
    q = (rf - dn) / (up - dn) 
    Suu = S0*up**2
    Sud = S0*up*dn
    Sdd = S0*dn**2
    Cuu = Suu - S0
    Pud = S0 - Sud
    Puu = S0 - Sdd
    Cu = (q * Cuu)/ (rf)
    C = (q * Cu ) / (rf)
    Pu = ((1-q) * Pud) / rf
    Pudd = ((q *  Pud) + (1-q)*(Puu))/ (rf)
    P = (Pu * q +(1-q) * ( Pudd)) / (rf)
    print(P)
task2a()


1.9208409406350115


# Task 3 (1 point)
Assume that now the price of stock is 100. In the next period price can be either 120 or 80, and you estimate
probability of an upmove as 0.7. Risk-free rate is 3% per period.
The one-year put price with strike 110 is 11.23. Check if it violates put-call parity. If it violates the parity, return
the fair price of the put option. If it does not violate the parity, return 0.0.


In [16]:
def task3(S0, Su, Sd, P0, K, r, p=0.7):
 """
 inputs:
 S0 = current stock price,
 Su = stock price the next period in the up state,
 Sd = stock price the next period in the down state,
 P0 = observed price of a put
 K = strike price
 r = interest rate per period as decimal (that is, 0.03 for 3%)
 p = probability of an up-move (0.7 by default)
 returns:
 If put violates the parity, return the fair price. 
 If it does not violate the parity, return 0.0.
 """
 ans = None
 ### BEGIN SOLUTION
 # solve the system
 # Delta * Su + B * (1+r) = Cu
 # Delta * Sd + B * (1+r) = Cd
 # => Delta = (Pu-Pd)/(Su-Sd)
 # B = ...
 rf = 1+r
 F = S0*rf # fair forward
 q = (F - Sd)/(Su-Sd) # Rn probability
 C = (q*max(Su-K,0) + (1-q)*max(Sd-K,0))/rf
 P = C - S0 + K/rf # put-call parity answer
 ans = 0.0 if np.isclose(P,P0) else P
 ### END SOLUTION
 return ans


In [17]:
S0, Su, Sd, P0, K, r = 100, 120, 80, 1, 110, 0.03
ans = task3(S0, Su, Sd, P0, K, r, p=0.7)
ans

12.37864077669903